<h3>Loading plain weights and running inference</h3>

In [1]:
import os
os.environ['ENV'] = 'prod'
os.environ['REGION'] = 'apse1'
os.environ['TENANT'] ="in"
os.environ['RECO_S3_BUCKET'] = "p13n-reco-offline-prod"
os.environ['COUNTRY_KEY']= "in"
os.environ['AWS_REGION']= "ap-southeast-1"
os.environ['USE_REAL_CMS3']= "True"
os.environ['RECO_CREDENTIAL']= "-----BEGINRSAPRIVATEKEY-----\nMGICAQACEQCdHOlGnxIMWCMzjK2JAg37AgMBAAECEGOIwGTEO9vd3X9+jyiF4NECCQnoqDakDgSm2QIID9sadWN0XvMCCQLiqPkgVKSuIQIIDCAsWM+pJB8CCQG0jbIGCNX9MA==\n-----ENDRSAPRIVATEKEY-----"

In [2]:
import argparse
import json
import os
import numpy as np
import s3fs
import pyarrow
import tensorflow as tf

tfv1 = tf.compat.v1
tfv1.disable_v2_behavior()

import tensorflow_addons as tfa
import tensorflow_recommenders_addons as tfra

from common.config.utils import data_path, model_path
from common.config import TENANT
from tpfy.tf_model.tpfy_model_v3_mtl import TpfyModelV3, TpfyMtlModelConfig
from tpfy.etl.schema import TpfyMtlDatasetSchema
from model.parquet_dataset import TFParquetDataset
from tpfy.common import TpfyDataPath
from omegaconf import OmegaConf
from dataclasses import dataclass
from tpfy.train_v3_mtl import make_example_mtl, TpfyTrainConfig, TpfyConfig

S3_TPFY_MODEL_EXPORT = model_path(TpfyDataPath.S3_TPFY_MODEL_EXPORT, TENANT)

def load_model_weights_from_s3(model_name, use_s3=True):
    """
    Load plain weights from S3 or local filesystem
    
    Args:
        model_name: Name of the model (e.g., "my_model/12345678")
        checkpoint: Specific checkpoint to load (None = read from checkpoint file)
        use_s3: If True, load from S3; if False, load from local export/
    
    Returns:
        dict: Plain weights dictionary
    """
    if use_s3:
        filesystem = s3fs.S3FileSystem(use_ssl=False)
        model_path = S3_TPFY_MODEL_EXPORT % model_name
    else:
        filesystem = pyarrow.LocalFileSystem()
        model_path = os.path.join("export", model_name)
    
    # Read checkpoint if not specified
    checkpoint_path = os.path.join(model_path, "checkpoint")
    print(f"Reading checkpoint from: {checkpoint_path}")

    if not filesystem.exists(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint file not found: {checkpoint_path}")

    with filesystem.open(checkpoint_path, "r") as f:
        checkpoint = f.read().strip()
    print(f"Using checkpoint: {checkpoint}")
    
    # Load weights
    weights_path = os.path.join(model_path, checkpoint, "plain_weights.npz")
    print(f"Loading weights from: {weights_path}")
    
    if not filesystem.exists(weights_path):
        raise FileNotFoundError(f"Weights file not found: {weights_path}")
    
    with filesystem.open(weights_path, "rb") as f:
        plain_weights = {}
        npz_data = np.load(f)
        for k, v in npz_data.items():
            plain_weights[k] = v
    
    print(f"Loaded {len(plain_weights)} weight tensors")
    print(f"Weight keys: {list(plain_weights.keys())[:10]}...")  # Show first 10
    
    return plain_weights

2026-02-07 17:00:23.146242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2026-02-07 17:00:23.146270: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
class Args:
    """Simple class to hold training arguments (replaces argparse)"""
    def __init__(self):
        # Positional arguments
        self.model_name = "tpfy-v3-mtl-r2"
        self.date = "2026-02-03"  # Training date
        self.val_date = "2026-02-03"  # Validation date
        
        # Optional arguments
        self.conf = None
        self.max_epoch = None
        self.val_days = 1
        self.click_ns = 0.08
        self.variant = "cms3"
        self.num_workers = 4
        self.repeat = 1
        self.eval_freq = None
        self.lr = 1e-4
        self.batch_size = 32
        self.click_weight = 1.0
        self.watch_weight = 1.0
        self.upload = False  # Set to False if you don't want to upload to S3
        self.reload_local_model = None
        self.reload_s3_model = "tpfy-v3-mtl-r2"  # Set to None if starting fresh
        self.extract_activations = True
        self.output = None
        self.clear_nn = False
        self.ckpt = None
        self.verbose = False
        self.countries = None

# Create args instance
args = Args()

# Display configuration
print("Training Configuration:")
print(f"  Model Name: {args.model_name}")
print(f"  Training Date: {args.date}")
print(f"  Validation Date: {args.val_date}")
print(f"  Variant: {args.variant}")
print(f"  Click NS: {args.click_ns}")
print(f"  Num Workers: {args.num_workers}")
print(f"  Reload Model: {args.reload_s3_model}")
print(f"  Upload: {args.upload}")

Training Configuration:
  Model Name: tpfy-v3-mtl-r2
  Training Date: 2026-02-03
  Validation Date: 2026-02-03
  Variant: cms3
  Click NS: 0.08
  Num Workers: 4
  Reload Model: tpfy-v3-mtl-r2
  Upload: False


In [4]:
# def run_inference(args):
print("="*80)
print("TPFY MODEL INFERENCE FROM PLAIN WEIGHTS")
print("="*80)

# Load configuration
config_name = f"tpfy/tpfy_config/mtl-{TENANT}.yaml"
if not os.path.exists(config_name):
    raise FileNotFoundError(f"Config file {config_name} not found")

hparams: TpfyConfig = OmegaConf.merge(
    OmegaConf.structured(TpfyConfig),
    OmegaConf.load(config_name),
)
print(f"\nLoaded config: {config_name}")

# Override batch size if specified
if args.batch_size:
    hparams.train.batch_size = args.batch_size

batch_size = hparams.train.batch_size
print(f"Batch size: {batch_size}")

# Load dataset
variant = args.variant
if variant and not variant.startswith("-"):
    variant = "-" + variant

data_date = args.date
data_path_str = data_path(
    TpfyDataPath.S3_TPFY_IMPR_V3_AGG_MTL_EXTRACTED_EXAMPLES_VAR, TENANT
) % (variant, data_date)

print(f"\nData path: {data_path_str}")

dataset = TFParquetDataset([data_path_str], TpfyMtlDatasetSchema, shuffle_files=False)
tf_dataset = dataset.create_tf_dataset(batch_size).map(make_example_mtl)

# if args.num_batches:
#     tf_dataset = tf_dataset.take(args.num_batches)
#     print(f"Limiting to {args.num_batches} batches")

# Create TensorFlow session
session = tfv1.keras.backend.get_session()

# Build model
print(f"\n{'='*80}")
print("BUILDING MODEL")
print(f"{'='*80}")

tpfy_model = TpfyModelV3(
    hparams.model,
    click_ns=args.click_ns,
    enable_random_watch=hparams.train.enable_random_watch,
)

# Create optimizer (needed for compilation, even though we won't train)
optimizer = tfa.optimizers.AdamW(
    weight_decay=0.0,  # Not needed for inference
    learning_rate=0.001,  # Not needed for inference
    epsilon=1e-4,
)
optimizer = tfra.dynamic_embedding.DynamicEmbeddingOptimizer(optimizer)

# Compile model (required to initialize variables)
from model.losses import masked_binary_entropy_loss
from model.metrics import MaskedAUC

loss_dict = {
    "click": masked_binary_entropy_loss(from_logits=True),
    "watch": masked_binary_entropy_loss(from_logits=True),
    "random_watch": masked_binary_entropy_loss(from_logits=False),
    "paywall_view": masked_binary_entropy_loss(from_logits=True),
    "add_watchlist": masked_binary_entropy_loss(from_logits=True),
}
metric_dict = {
    "click": MaskedAUC(from_logits=True),
    "watch": MaskedAUC(from_logits=True),
    "random_watch": MaskedAUC(from_logits=False),
    "paywall_view": MaskedAUC(from_logits=True),
    "add_watchlist": MaskedAUC(from_logits=True),
}

tpfy_model.compile(optimizer=optimizer, loss=loss_dict, metrics=metric_dict)
print("Model compiled")

# ========== BUILD MODEL WITH SAMPLE DATA ==========
print("\nFetching sample batch to build model...")
iterator = tf_dataset.make_one_shot_iterator()
next_batch = iterator.get_next()

sample_features, sample_labels, sample_metadata = session.run(next_batch)
print(f"Sample batch loaded: {len(sample_features)} features")

print("Running model forward pass to create weights...")
_ = tpfy_model(sample_features, training=False)

print(f"Model built successfully")
print(f"Total trainable variables: {len(tpfy_model.trainable_variables)}")

# Initialize all variables
print("\nInitializing TensorFlow variables...")
session.run([
    tfv1.global_variables_initializer(),
    tfv1.local_variables_initializer(),
    tfv1.tables_initializer(),
])
print("✅ Variables initialized")
# ==================================================

# Load plain weights from S3
print(f"\n{'='*80}")
print("LOADING MODEL WEIGHTS")
print(f"{'='*80}")

plain_weights = load_model_weights_from_s3(
    args.model_name,
    use_s3=True
)
plain_weights_modified = {k.replace('train/', ''): v for k, v in plain_weights.items()}

# Restore weights (NOW the model is built, so this will work)
print("\nRestoring model weights...")
restore_ops = tpfy_model.restore_plain_weights_ops(
    plain_weights_modified,
    clear_nn=args.clear_nn
)
session.run(restore_ops)
print("Weights restored successfully")

# Create NEW iterator (reset to start of dataset)
print(f"\n{'='*80}")
print("RUNNING INFERENCE")
print(f"{'='*80}")

iterator = tf_dataset.make_one_shot_iterator()
next_batch = iterator.get_next()

# Get compress_output tensor (linear_input)
graph = tf.compat.v1.get_default_graph()

# Find the compress_out tensor that is fed to output layer that predicts click / watch
compress_output_tensor = graph.get_tensor_by_name('tpfy_model_v3/deepfm/Relu:0')

TPFY MODEL INFERENCE FROM PLAIN WEIGHTS

Loaded config: tpfy/tpfy_config/mtl-in.yaml
Batch size: 32

Data path: s3://p13n-reco-offline-prod/dataset_v5/tpfy-impr-v3/agg-mtl-extracted-cms3/2026-02-03
files s3://p13n-reco-offline-prod/dataset_v5/tpfy-impr-v3/agg-mtl-extracted-cms3/2026-02-03/part-00000-tid-2481856773249800027-feb475eb-c37d-4ca2-8331-01bcc65d4598-2309-1-c000.snappy.parquet,s3://p13n-reco-offline-prod/dataset_v5/tpfy-impr-v3/agg-mtl-extracted-cms3/2026-02-03/part-00001-tid-2481856773249800027-feb475eb-c37d-4ca2-8331-01bcc65d4598-2407-1-c000.snappy.parquet,s3://p13n-reco-offline-prod/dataset_v5/tpfy-impr-v3/agg-mtl-extracted-cms3/2026-02-03/part-00002-tid-2481856773249800027-feb475eb-c37d-4ca2-8331-01bcc65d4598-2522-1-c000.snappy.parquet,s3://p13n-reco-offline-prod/dataset_v5/tpfy-impr-v3/agg-mtl-extracted-cms3/2026-02-03/part-00003-tid-2481856773249800027-feb475eb-c37d-4ca2-8331-01bcc65d4598-2471-1-c000.snappy.parquet,s3://p13n-reco-offline-prod/dataset_v5/tpfy-impr-v3/agg-

2026-02-07 17:00:24.631466: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2026-02-07 17:00:24.631489: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2026-02-07 17:00:24.631503: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-11-72-173): /proc/driver/nvidia/version does not exist
2026-02-07 17:00:24.631661: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.




BUILDING MODEL
Model compiled

Fetching sample batch to build model...
Instructions for updating:
This is a deprecated API that should only be used in TF 1 graph mode and legacy TF 2 graph mode available through `tf.compat.v1`. In all other situations -- namely, eager mode and inside `tf.function` -- you can consume dataset elements using `for elem in dataset: ...` or by explicitly creating iterator via `iterator = iter(dataset)` and fetching its elements via `values = next(iterator)`. Furthermore, this API is not available in TF 2. During the transition from TF 1 to TF 2 you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)` to create a TF 1 graph mode style iterator for a dataset created through TF 2 APIs. Note that this should be a transient state of your code base as there are in general no guarantees about the interoperability of TF 1 and TF 2 code.


2026-02-07 17:00:25.287038: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2499995000 Hz
2026-02-07 17:00:25.334341: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Sample batch loaded: 9 features
Running model forward pass to create weights...
--------------
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
q Tensor("tpfy_model_v3/feature_prep/strided_slice:0", shape=(32, 32), dtype=float32)
k Tensor("tpfy_model_v3/feature_prep/watched_content_embedding_unpooled:0", shape=(32, ?, 32), dtype=float32)
Kw Tensor("tpfy_model_v3/feature_prep/GetSlotFids:1", shape=(32, ?), dtype=float32)
target embedding shape (32, 9, 32)
user embedding shape (32, 27, 32)
target: Tensor("tpfy_model_v3/feature_prep/target_feature/target_embeddings:0", shape=(32, 9, 32), dtype=float32)
user: Tensor("tpfy_model_v3/feature_prep/user_feature/user_embeddings:0", shape=(32, 27, 32), dtype=float32)
watched: Tensor("tpfy_model_v3/feature_prep/dot_prod_attention_pooling/cond/Merge:0", shape=(32, 32), dtype=float32)
fm_user Tensor("tpfy_model_v3/deepfm/fwfm/concat:0", shape=(32, 28, 32), dtype

2026-02-07 17:00:32.109251: I ./tensorflow_recommenders_addons/dynamic_embedding/core/kernels/lookup_impl/lookup_table_op_cpu.h:143] HashTable on CPU is created on optimized mode: K=l, V=f, DIM=32, init_size=8192


<h3>Run inference</h3>

In [5]:
A_inv = np.load('A_inv.npy')
b = np.load('b.npy')
theta = np.load('theta.npy')

In [ ]:
from tqdm import tqdm

def get_activations_and_labels(iterator, model, last_layer_tensor):
    features, labels, metadata = session.run(iterator)

    # Run model
    predictions = model(features, training=False)

    # Execute
    pred_values, activation_values = session.run(
        [predictions, last_layer_tensor]
    )

    return activation_values, labels, pred_values, metadata

def validation(iterator, model, last_layer_tensor, d=128, runs=10):
    predictions = []
    labels = []
    variances = []
    deepFM_predictions = []
    content_ids = []
    masks = []
    
    for i in tqdm(range(runs)):
        H, y_batch_all_labels, pred_values, metadata = get_activations_and_labels(iterator, model, last_layer_tensor)
        y_batch = y_batch_all_labels['click']
        pred_values = pred_values['click']
        
        mask = (y_batch != -1)
        
        if not np.any(mask):
            continue
            
        H = H[mask.squeeze()]
        # pred_values = pred_values[mask.squeeze()]
        y_batch = y_batch[mask].reshape(sum(mask)[0], 1)
        pred_values = pred_values[mask].reshape(sum(mask)[0], 1)
        
        H = H / (np.linalg.norm(H, axis=1, keepdims=True) + 1e-8)
        variance = np.sqrt(H @ A_inv @ H.T)

        predictions.append(H@theta)
        labels.append(y_batch)
        variances.append(variance)
        deepFM_predictions.append(pred_values)
        content_ids.append([content_id for content_id, mask_bool in zip(metadata['content_id'], mask) if mask_bool])

    return predictions, labels, variances, deepFM_predictions, content_ids

predictions, labels, variances, deepFM_predictions, content_ids = validation(next_batch, tpfy_model, compress_output_tensor, runs = 10_000)

  0%|                                                                                                                                                            | 0/10000 [00:00<?, ?it/s]

--------------
q Tensor("tpfy_model_v3_1/feature_prep/strided_slice:0", shape=(32, 32), dtype=float32)
k Tensor("tpfy_model_v3_1/feature_prep/watched_content_embedding_unpooled:0", shape=(32, ?, 32), dtype=float32)
Kw Tensor("tpfy_model_v3_1/feature_prep/GetSlotFids:1", shape=(32, ?), dtype=float32)
target embedding shape (32, 9, 32)
user embedding shape (32, 27, 32)
target: Tensor("tpfy_model_v3_1/feature_prep/target_feature/target_embeddings:0", shape=(32, 9, 32), dtype=float32)
user: Tensor("tpfy_model_v3_1/feature_prep/user_feature/user_embeddings:0", shape=(32, 27, 32), dtype=float32)
watched: Tensor("tpfy_model_v3_1/feature_prep/dot_prod_attention_pooling/cond/Merge:0", shape=(32, 32), dtype=float32)
fm_user Tensor("tpfy_model_v3_1/deepfm/fwfm/concat:0", shape=(32, 28, 32), dtype=float32)
fm_item Tensor("tpfy_model_v3_1/feature_prep/target_feature/target_embeddings:0", shape=(32, 9, 32), dtype=float32)
fwfm out Tensor("tpfy_model_v3_1/deepfm/fwfm/Reshape:0", shape=(32, 252), dtyp

/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in sqrt
  0%|                                                                                                                                                 | 1/10000 [00:05<14:16:45,  5.14s/it]

--------------
q Tensor("tpfy_model_v3_2/feature_prep/strided_slice:0", shape=(32, 32), dtype=float32)
k Tensor("tpfy_model_v3_2/feature_prep/watched_content_embedding_unpooled:0", shape=(32, ?, 32), dtype=float32)
Kw Tensor("tpfy_model_v3_2/feature_prep/GetSlotFids:1", shape=(32, ?), dtype=float32)


  0%|                                                                                                                                                  | 2/10000 [00:05<6:27:20,  2.32s/it]

...

--------------
q Tensor("tpfy_model_v3_112/feature_prep/strided_slice:0", shape=(32, 32), dtype=float32)
k Tensor("tpfy_model_v3_112/feature_prep/watched_content_embedding_unpooled:0", shape=(32, ?, 32), dtype=float32)
Kw Tensor("tpfy_model_v3_112/feature_prep/GetSlotFids:1", shape=(32, ?), dtype=float32)
target embedding shape (32, 9, 32)
user embedding shape (32, 27, 32)
target: Tensor("tpfy_model_v3_112/feature_prep/target_feature/target_embeddings:0", shape=(32, 9, 32), dtype=float32)
user: Tensor("tpfy_model_v3_112/feature_prep/user_feature/user_embeddings:0", shape=(32, 27, 32), dtype=float32)
watched: Tensor("tpfy_model_v3_112/feature_prep/dot_prod_attention_pooling/cond/Merge:0", shape=(32, 32), dtype=float32)
fm_user Tensor("tpfy_model_v3_112/deepfm/fwfm/concat:0", shape=(32, 28, 32), dtype=float32)
fm_item Tensor("tpfy_model_v3_112/feature_prep/target_feature/target_embeddings:0", shape=(32, 9, 32), dtype=float32)
fwfm out Tensor("tpfy_model_v3_112/deepfm/fwfm/Reshape:0", sha

  1%|█▌                                                                                                                                              | 112/10000 [01:27<3:29:45,  1.27s/it]

--------------
q Tensor("tpfy_model_v3_113/feature_prep/strided_slice:0", shape=(32, 32), dtype=float32)
k Tensor("tpfy_model_v3_113/feature_prep/watched_content_embedding_unpooled:0", shape=(32, ?, 32), dtype=float32)
Kw Tensor("tpfy_model_v3_113/feature_prep/GetSlotFids:1", shape=(32, ?), dtype=float32)
target embedding shape (32, 9, 32)
user embedding shape (32, 27, 32)
target: Tensor("tpfy_model_v3_113/feature_prep/target_feature/target_embeddings:0", shape=(32, 9, 32), dtype=float32)
user: Tensor("tpfy_model_v3_113/feature_prep/user_feature/user_embeddings:0", shape=(32, 27, 32), dtype=float32)
watched: Tensor("tpfy_model_v3_113/feature_prep/dot_prod_attention_pooling/cond/Merge:0", shape=(32, 32), dtype=float32)
fm_user Tensor("tpfy_model_v3_113/deepfm/fwfm/concat:0", shape=(32, 28, 32), dtype=float32)
fm_item Tensor("tpfy_model_v3_113/feature_prep/target_feature/target_embeddings:0", shape=(32, 9, 32), dtype=float32)
fwfm out Tensor("tpfy_model_v3_113/deepfm/fwfm/Reshape:0", sha

  1%|█▋                                                                                                                                              | 113/10000 [01:28<3:32:12,  1.29s/it]

--------------
q Tensor("tpfy_model_v3_114/feature_prep/strided_slice:0", shape=(32, 32), dtype=float32)
k Tensor("tpfy_model_v3_114/feature_prep/watched_content_embedding_unpooled:0", shape=(32, ?, 32), dtype=float32)
Kw Tensor("tpfy_model_v3_114/feature_prep/GetSlotFids:1", shape=(32, ?), dtype=float32)
target embedding shape (32, 9, 32)
user embedding shape (32, 27, 32)
target: Tensor("tpfy_model_v3_114/feature_prep/target_feature/target_embeddings:0", shape=(32, 9, 32), dtype=float32)
user: Tensor("tpfy_model_v3_114/feature_prep/user_feature/user_embeddings:0", shape=(32, 27, 32), dtype=float32)
watched: Tensor("tpfy_model_v3_114/feature_prep/dot_prod_attention_pooling/cond/Merge:0", shape=(32, 32), dtype=float32)
fm_user Tensor("tpfy_model_v3_114/deepfm/fwfm/concat:0", shape=(32, 28, 32), dtype=float32)
fm_item Tensor("tpfy_model_v3_114/feature_prep/target_feature/target_embeddings:0", shape=(32, 9, 32), dtype=float32)
fwfm out Tensor("tpfy_model_v3_114/deepfm/fwfm/Reshape:0", sha

In [ ]:
all_labels = np.concatenate(labels, axis=None)
all_predictions = np.concatenate(predictions, axis=None)
all_deepFMpredictions = np.concatenate(deepFM_predictions, axis=None)
all_variances = np.concatenate([np.diag(x) for x in variances], axis = None)
all_contentIds = [x for content_id in content_ids for x in content_id]

In [ ]:
len(all_variances), len(all_labels), len(all_predictions), len(all_deepFMpredictions), len(all_contentIds)

In [ ]:
import pandas as pd
content_popularity_tag = pd.read_csv('/home/ubuntu/vedansh/code/rank-change-analysis/content_id_popularity_tag.csv')
content_popularity_tag.drop_duplicates(subset=['sub_title_id'], inplace = True)
content_popularity_tag.reset_index(drop = True, inplace = True)
content_popularity_dict = content_popularity_tag.set_index('sub_title_id')['popularity_tag'].to_dict()

<h3>Popularity tier VS deepFM scores</h3>

In [ ]:
#content popularity tier vs deepfmscores
content_deepFM_scores = {int(content_id): deepFMscore for content_id, deepFMscore in zip(all_contentIds, all_deepFMpredictions)}
content_deepFM_scores_df = pd.DataFrame(content_deepFM_scores, index = [0]).T.reset_index().rename(columns = {'index': 'content_id', 0: 'deepFMscore'})
content_deepFM_scores_df['popularity_tag'] = content_deepFM_scores_df['content_id'].apply(lambda x: content_popularity_dict.get(int(x)))

In [ ]:
content_deepFM_scores_df.groupby('popularity_tag')['deepFMscore'].describe()

In [ ]:
content_deepFM_scores_df.groupby('popularity_tag')['deepFMscore'].describe().to_csv('popularity_tag_deepFM_score_stats.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'df' is your DataFrame
plt.figure(figsize=(10, 6))

# Plotting the distribution (KDE plot)
sns.kdeplot(
    data=content_deepFM_scores_df, 
    x='deepFMscore', 
    hue='popularity_tag', 
    fill=True, 
    common_norm=False, 
    # palette='viridis', 
    alpha=0.5, 
    linewidth=2
)

plt.title('Distribution of deepFMscore$ by Popularity Tag')
plt.xlabel('$deepFMscore$')
plt.ylabel('Density')
plt.grid(True, linestyle='--', alpha=0.6)

# Save the plot
plt.savefig('distplot_deepFMscore.png')

<h3>Popularity tier VS linUCB variances</h3>

In [ ]:
#content popularity tier vs variancess
content_variances = {int(content_id): variances for content_id, variances in zip(all_contentIds, all_variances)}
content_variances_df = pd.DataFrame(content_variances, index = [0]).T.reset_index().rename(columns = {'index': 'content_id', 0: 'variances'})
content_variances_df['popularity_tag'] = content_variances_df['content_id'].apply(lambda x: content_popularity_dict.get(int(x)))
content_variances_df.groupby('popularity_tag')['variances'].describe()

In [ ]:
content_variances_df.groupby('popularity_tag')['variances'].describe().to_csv('popularity_tag_lunUCB_variances_stats.csv')

In [ ]:
# Assuming 'df' is your DataFrame
plt.figure(figsize=(10, 6))

# Plotting the distribution (KDE plot)
sns.kdeplot(
    data=content_variances_df, 
    x='variances', 
    hue='popularity_tag', 
    fill=True, 
    common_norm=False, 
    # palette='viridis', 
    alpha=0.5, 
    linewidth=2
)

plt.title('Distribution of variances by Popularity Tag')
plt.xlabel('$variances$')
plt.ylabel('Density')
plt.grid(True, linestyle='--', alpha=0.6)

# Save the plot
plt.savefig('popularity_tag_lunUCB_variances.png')

<h3>Popularity tier VS linUCB mean</h3>

In [ ]:
#content popularity tier vs variancess
content_predictions = {int(content_id): predictions for content_id, predictions in zip(all_contentIds, all_predictions)}
content_predictions_df = pd.DataFrame(content_predictions, index = [0]).T.reset_index().rename(columns = {'index': 'content_id', 0: 'predictions'})
content_predictions_df['popularity_tag'] = content_predictions_df['content_id'].apply(lambda x: content_popularity_dict.get(int(x)))
content_predictions_df.groupby('popularity_tag')['predictions'].describe()

In [ ]:
content_predictions_df.groupby('popularity_tag')['predictions'].describe().to_csv('popularity_tag_lunUCB_means_stats.csv')

In [ ]:
plt.figure(figsize=(10, 6))

# Plotting the distribution (KDE plot)
sns.kdeplot(
    data=content_predictions_df, 
    x='predictions', 
    hue='popularity_tag', 
    fill=True, 
    common_norm=False, 
    # palette='viridis', 
    alpha=0.5, 
    linewidth=2
)

plt.title('Distribution of $predictions$ by Popularity Tag')
plt.xlabel('$predictions$')
plt.ylabel('Density')
plt.grid(True, linestyle='--', alpha=0.6)

# Save the plot
plt.savefig('popularity_tag_lunUCB_mean.png')

In [ ]:
print('s')